# ch05 프록시와 옵저버 패턴으로 객체 간 통신하기

- 규모가 큰 객체를 가지고 작업을 할 때
- 사용하는 순간까지 생성을 미뤄두는 것이 메모리와 시간을 저약
- 이런 객체를 생성했을 때, 요청이 새로 들어올 때마다 다시 생성하지 않고, 기존에 생성해 놓았던 객체를 사용하고 새로운 참조를 만드는 것이 좋음
- 이 객체에 대한 사용이 끝났을 때는 가능한 빨리 메모리를 정리해줘야 함
- 따라서 규모가 큰 객체에 대한 참조 카운트가 필요하고, 이것을 구현하기 위해 중간자 역할을 하는 무언가가 필요함
- 이 문제에 대한 해결책으로 proxy 가 있음
- 프록시는 클라이언트 코드와 클라이언트 코드가 사용하는 객체를 분리하는 디자인 패턴
- 클라이언트는 실제 객체처럼 동작하는 대리 프록시 객체를 사용하고, 대리 객체는 실제 객체를 호출함
- 게으른 초기화(lazy initialization)를 실제로 필요한 순간까지 미루는 것. 하지만 이 경우말고도 프록시를 활요하는 때가 있음
- 프록시는 로깅 구현, 네트워크 연결, 공유 객체 접근 제어, 참조 카운트 구현 등 여러 경우에 유용한 수단

## 프록시 디자인 패턴

- 프록시: 자신과 동일한 인터페이스를 가지고 있는 클래스에 대한 인터페이스 역할을 하는 클래스
- 프록시는 네트워크 연결, 메모리 규모가 큰 객체, 파일 복사가 불가능한 리소스 등과 함께 인터페이스 역할을 수행함

### 프록시 패턴이 해결하는 문제

- 프록시는 다른 객체에 대한 플레이스 홀더를 제공해서 이에 대한 접근을 조절함
- 프록시는 추가적 수준의 간접 접근(indirection)을 사용해서 분산, 조절, 영리한 접근을 지원함
- 프록시는 실제 부속품이 지나치게 복잡해지는 것을 방지하기 위해 래퍼와 델리게이션을 추가함

### 프록시 패턴의 사용

- 다른 객체에 대한 접근 제어를 위해. 예를 들어 보안상의 필요성을 들 수 있음
- Subject에 대한 모든 호출과 그 파라미터를 기록하기 위해
- 원격 머신이나 주소가 다른 공간에 있는 Subject에 연결하기 위해서. Proxy는 원격 객체에 대한 인터페이스를 가지고 있지만 호출자에게 투명한 연결 루틴을 처리하기도 함
- 필요한 경우에만 규모가 큰 객체를 인스턴스화하기 위해서. 또한 이 객체나 그 일부를 캐싱할 수도 있음
- 결과를 공유하는 여러 클라이언트에 계산 값을 반환하기 전에 임시로 저장하기 위해서
- 객체에 대한 참조를 세기 위해서

### 프록시 패턴의 장점과 단점

- 자주 사용하는 객체를 캐싱함으로써 애플리케이션의 성능을 최적화
- Proxy에서 접근 권한을 확인하고, 권한이 있는 경우에만 RealSubject로 delegate 하면 애플리케이션의 보안 수준이 높아짐
- 원격 시스템 간 상호작용을 넣을 때, 프록시가 네트워크 연결과 전송 루틴을 책임지고 원격 객체에 대한 호출을 위임함
- 프록시 패턴을 사용하면 간혹 객체로부터 응답이 느려지기도 함
- 예를 들어 lazy initialization 에 프록시를 사용하고 객체를 처음으로 요청하면, 초기화에 걸리는 시간 때문에 응답 시간이 더 길어질 수 있음

### 파이썬 구현

- 천만 자릿수를 포함하는 거대한 객체 RealSubject를 인스턴스화 해야 함
- 인스턴스화 과정에 어느 정도의 시간과 RAM 공간이 필요하기 때문에, 인터페이스를 제공하는 프록시를 만들 것
- 메타 클래스 ABCMeta와 데코레이터 abstractmethod를 임포트한다. 이들은 추상 클래스를 구현하는데 사용함
- 추상 클래스는 구현이 생략된 메소드를 가지고 있는 클래스를 의미함
- 추상 클래스는 인스턴스화 할 수 없고, 이를 상속받은 클래스 역시 abstractmethod 데코레이터로 표시한 메소드를 구현하지 않는다면 인스턴스화가 불가능함
- 따라서, 추상 클래스를 구현하고 싶다면 ABCMeta에 \_\_metaclass\_\_ 속성을 할당하고 구현이 빠진 메소드에 abstractmethod 데코레이터를 붙여야 함
- 우선 RealSubject와 그 프록시에 인터페이스를 제공하는 추상 클래스를 만들어 보자

In [1]:
from abc import ABCMeta, abstractmethod
import random


class AbstractSubject(object):
    """실제와 프록시 객체에 대한 일반적인 인터페이스"""
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def sort(self, reverse=False):
        pass

In [2]:
class RealSubject(AbstractSubject):
    """인스턴스화 하는데 많은 시간과 메모리를 차지하는 커다란 객체에 대한 클래스"""
    def __init__(self):
        self.digits = [random.random() for i in xrange(10000000)]
        
    def sort(self, reverse=False):
        self.digits.sort()
        if reverse:
            self.digits.reverse()

In [3]:
# getattr(object, name[, default]) -> value
getattr?

In [4]:
class Proxy(AbstractSubject):
    """RealSubject와 동일한 인터페이스를 갖고 있는 프록시"""
    reference_count = 0
    
    def __init__(self):
        """기존에 생성된 것이 있으면 캐시 해놓고, 없다면 생성하는 생성자"""
        if not getattr(self.__class__, 'cached_object', None):
            self.__class__.cached_object = RealSubject()
        else:
            print('Using cached object')
            
        self.__class__.reference_count += 1
        print('Count of references = ', self.__class__.reference_count)
              
    def sort(self, reverse=False):
        """인자는 프록시에 의해 기록된다"""
        print('Called sort method with args:')
        print(locals().items())
        
        self.__class__.cached_object.sort(reverse=reverse)
        
    def __del__(self):
        """객체에 대한 참조를 감소시킨다. 만약 참조가 0이 되면 객체르 삭제한다"""
        self.__class__.reference_count -= 1
        if self.__class__.reference_count == 0:
            print('Number of reference_count is 0. Deleting cached object...')
            del self.__class__.cached_object
        print('Deleted object. Count of objects = ', 
              self.__class__.reference_count)

In [5]:
if __name__ == '__main__':
    proxy1 = Proxy()
    print('')
    
    proxy2 = Proxy()
    print('')
    proxy3 = Proxy()
    print('')
    
    proxy1.sort(reverse=True)
    print('')
    
    print('Deleting proxy2')
    del proxy2
    print('')
    
    print('The other objects are deleted upon program termination')

('Count of references = ', 1)

Using cached object
('Count of references = ', 2)

Using cached object
('Count of references = ', 3)

Called sort method with args:
[('self', <__main__.Proxy object at 0x105f3bcd0>), ('reverse', True)]

Deleting proxy2
('Deleted object. Count of objects = ', 2)

The other objects are deleted upon program termination


In [33]:
def sort(digits, reverse=False):
#     digits.sort()
    if reverse:
        digits[::-1]
#         digits.reverse()
    return digits
    
l = [5, 3, 1, 4, 9, 6]
print(sort(l))
#

[5, 3, 1, 4, 9, 6]


## 옵저버 디자인 패턴

- 소프트웨어 공학에서 일대다(one-to-many) 관계를 활용할 수 있게 함
- 일대다 관계가 필요한 상황은 많이 있음
- 블로그를 구독하는 여러 독자들, 사용자 인터페이스 아이템을 클릭했을 때 동시에 반응하는 여러 이벤트 리스너들, 인터넷에서 데이터를 받았을 때 알림을 보내는 폰 애플리케이션 등이 이에 해당

### 유사한 점

- 구독자는 신문 구독을 신청한다
- 신문을 구독한다
- 다른 사람도 이 신문을 구독할 수 있다
- 새로운 신문이 발행하면 모든 구독자는 새로운 신문을 받는다
- 더 이상 신문이 필요하지 않으면 구독을 취소하고, 그 이후로는 신문이 배달되지 않는다(하지만 다른 사람은 여전히 신문을 받는다)

- 발행-구독(publishing-subscriber) 패턴의 일종
- 소프트웨어 공학에서는 신문과는 다르게 자원, 병렬 스레드 신호, 운영체제 신호 등 어떠한 정보도 사용할 수 있음
- 이 정보는 구독자에게 전달되어야 하고, 옵저버 디자인 패턴은 구독과 배달을 관리해야 함
- 옵저버 패턴에서, 주체를 호출하는 객체는 옵저버를 호출하는 다른 객체 집합을 보유하고, 상태가 변하거나 할 때 그들의 메소드 중 하나를 호출해서 이를 알려야 함
- Subject는 시스템의 static 부분
- 애플리케이션이 실행되는 동안 단 하나의 Subject만 존재하는데, 이와 반대로 Observer는 변동이 가능한 부분
- 옵저버는 하나도 없을 수도, 여러 개가 될 수도 있으며, 이 값은 애플리케이션이 실행되는 동안 변함. 자주 갱신되는 부분은 일반적으로 Observer로 구현됨

### 옵저버 패턴이 해결하는 문제

- 특정 객체가 상태를 바꾸고, 이 변화에 따라 하나 혹은 그 이상의 객체가 그들의 상태를 바꿔야 하는 상황에 옵저버 패턴을 구현해서 객체간 의존도를 낮출 수 있음
- 실생활에서 찾을 수 있는 예로 트위터 같은 마이크로 블로그를 들 수 있다. 새로운 트윗을 남기면(자신 피드의 상태가 변경), 모든 팔로워(옵저버)에게 알림 메시지가 가고 타임라인에 방금 남긴 트윗이 갱신되어 나타남

### 옵저버 패턴 사용

- 옵저버 패턴은 하나의 객체가 변경되었을 때, 몇 개인지 모르는 나머지 객체도 이에 맞추어 변경되어야 하는 경우에 사용함

### 옵저버 패턴의 장점

- Subject와 Observer의 연결이 느슨하도록 유지
- Subject는 오직 Observer의 리스트만 알고 있고, 실제 클래스나 자세한 구현 등에 대해선 관심을 갖지 않음
- Subject와 Observer 간에 메시지 브로드캐스팅을 할 수 있음
- Observer의 수를 실행 시간에 바꿀 수 있음
- Subject는 어떠한 수의 Observer도 가질 수 있음

### 파이썬 구현

- 옵저버를 추가, 제거하고 알림 기능이 있는 간단한 서브젝트를 만들어 보자
- 모든 옵저버의 notify 메소드는 유닉스 타임스탬프를 얻고 미국 서식(12시)이나 유럽 서식(24시)으로 출력함
- self.observers에 옵저버 리스트를 넣어두고 register_observer와 unregister_observer 메소드로 옵저버를 추가하거나 제거하는 Subject를 만든다.
- 모든 옵저버에 정보를 보내고 싶으면 간단히 notify_observers 메소드를 호출하고 이 메소드는 모든 옵저버에 새로운 유닉스 타임스탬프 값을 전달함

In [47]:
import time


class Subject(object):
    def __init__(self):
        self.observers = []
        self.cur_time = None
        
    def register_observer(self, observer):
        if observer in self.observers:
            print(observer, 'already in subscribed observers')
        else:
            self.observers.append(observer)
            
    def unregister_observer(self, observer):
        try:
            self.observers.remove(observer)
        except ValueError:
            print('No such observer in subject')
            
    def notify_observers(self):
        self.cur_time = time.time()
        for observer in self.observers:
            observer.notify(self)

In [35]:
from abc import ABCMeta, abstractmethod
import datetime


class Observer(object):
    """Abstract class for observers, provides notify method as interface for subjects."""
    
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def notify(self, unix_timestamp):
        pass

In [49]:
class USATimeObserver(Observer):
    def __init__(self, name):
        self.name = name
        
    def notify(self, obj):
        time = datetime.datetime.fromtimestamp(
            int(obj.cur_time)).strftime('%Y-%m-%d %I:%M:%S%p')
        print('Observer', self.name, 'says:', time)
        

class EUTimeObserver(Observer):
    def __init__(self, name):
        self.name = name
        
    def notify(self, obj):
        time = datetime.datetime.fromtimestamp(
            int(obj.cur_time)).strftime('%Y-%m-%d %H:%M:%S')
        print('Observer', self.name, 'says:', time)

In [50]:
if __name__ == '__main__':
    subject = Subject()
    
    print('Adding usa_time_observer')
    observer1 = USATimeObserver('usa_time_observer')
    subject.register_observer(observer1)
    subject.notify_observers()
    
    time.sleep(2)
    print('Adding eu_time_observer')
    observer2 = EUTimeObserver('eu_time_observer')
    subject.register_observer(observer2)
    subject.notify_observers()
    
    time.sleep(2)
    print('Removing usa_time observer')
    subject.unregister_observer(observer1)
    subject.notify_observers()

Adding usa_time_observer
('Observer', 'usa_time_observer', 'says:', '2015-09-27 09:15:52AM')
Adding eu_time_observer
('Observer', 'usa_time_observer', 'says:', '2015-09-27 09:15:54AM')
('Observer', 'eu_time_observer', 'says:', '2015-09-27 09:15:54')
Removing usa_time observer
('Observer', 'eu_time_observer', 'says:', '2015-09-27 09:15:56')


In [42]:
time.time()

1443311925.874013

In [41]:
datetime.datetime.fromtimestamp(int(time.time()))

datetime.datetime(2015, 9, 27, 8, 58, 40)

In [43]:
datetime.datetime.fromtimestamp(int(time.time())).strftime('%Y-%m-%d %H:%M:%S')

'2015-09-27 08:58:57'

In [45]:
datetime.datetime.fromtimestamp(int(time.time())).strftime('%Y-%m-%d %I:%M:%S %p')

'2015-09-27 08:59:59 AM'

- 옵저버는 서브젝트의 메소드를 호출하고 속성에 접근해서 필요한 정보를 얻음
- 또다른 방식은 데이터를 아예 전달하지 않는 것. 그 옵저버는 어떤 이벤트가 발생하는 이에 대한 알림을 받고 여기에서 갱신된 값을 얻고 원하는 대로 해석하는 책임을 진다.

## 요약

- 프록시는 프록시와 동일한 인터페이스를 갖고있는 클래스에 대해 인터페이스 기능을 하는 클래스
- 클라이언트 코드는 인스턴스화 하고 프록시와 직접 작업하는데 비해, 프록시는 클라이언트 클래스에 대한 실제 작업을 위임함. 프록시를 사용하는 곳은 아주 많은데, 특히 캐싱, 참조 카운트, 접근 권한 제어 등에 잘 맞음
- 프록시 사용자는 응답 시간이 길어진다는 점에 주의해야 함
- 옵저버 디자인 패턴은 one-to-many 관계를 구현할 때 사용함
- 예를 들어 동일한 정보를 옵저버라 불리는 여러 리스너에게 전파하려는 경우가 여기 포함됨
- 옵저버 디자인 패턴은 서브젝트와 옵저버 간에 느슨한 연결을 유지시키는데, 서브젝트 측에서 옵저버에 대해 알아야 할 것은 알림을 위해 호출하는 메소드인 인터페이스 뿐
- 옵저버의 숫자는 정해지지 않고 실행하는 동안 변할 수 있음
- 커맨드 디자인 패턴이 어떻게 하나의 객체에서 호출을 캡슐화하는지, 이 디자인 패턴으로 실행 취소, 히스토리, 매크로 기능을 어떻게 구현하는지 배움